In [ ]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
def clean_text(text): 
    return text.lower()

In [ ]:
# review_data= pd.read_csv("C:\\AmazonReviews\\Reviews.csv")
df = pd.read_csv('original 2.csv', encoding='cp1252')
df=pd.DataFrame(df.values)
df.rename(columns = {0:'IDUPA',1:'EventDate',2:'Hospitalized',3:'Amputation',4:'Nature',5:'NatureTitle',6:'Part of Body',7:'Part of Body Title',8:'Event',9:'EventTitle',10:'Source',11:'SourceTitle'}, inplace = True)
review_data=df
review_data=review_data["Part of Body Title"]


In [ ]:
df.dropna()
df

In [ ]:
from dateutil import parser
df['time_stamp'] = df['EventDate'].map(parser.parse)
df['event_year'] = df['time_stamp'].map(lambda x: x.year)
df['event_month'] = df['time_stamp'].map(lambda x: x.month)
df['event_weekday'] = df['time_stamp'].map(lambda x: x.dayofweek)

In [ ]:
x=set(df['Hospitalized'].values)
x


In [ ]:
hos_ind = df[ (df['Hospitalized'] != 0) & (df['Hospitalized'] != 1) ].index
len(hos_ind)
df.drop(hos_ind , inplace=True)
df.head(5)

x=set(df['Amputation'].values)
x
hos_ind = df[ (df['Amputation'] != 0) & (df['Amputation'] != 1) ].index
len(hos_ind)
df.drop(hos_ind , inplace=True)
df

In [ ]:
print(df.Hospitalized.value_counts())
print(df.Amputation.value_counts())

# special

In [ ]:
# # Separate majority and minority classes
# from sklearn.utils import resample

# df_minority = df.loc[df.Hospitalized == 0].copy()
# df_majority = df.loc[df.Hospitalized == 1].copy()
# print(df_minority.shape)
# print(df_majority.shape)
# print(df.shape)


In [ ]:
# print(df.Hospitalized.value_counts())

In [ ]:
# # Upsample minority class
# df_minority_upsampled = resample(df_minority,
#                              replace=True,  # sample with replacement
#                              n_samples=58097,  # to match majority class
#                              random_state=42)  # reproducible results

# # Combine majority class with upsampled minority class
# df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# # Display new class counts
# print(df_upsampled.Hospitalized.value_counts())
# print(df_upsampled.Amputation.value_counts())
# df=df_upsampled

In [ ]:
# print(df.Hospitalized.value_counts())
# print(df.Amputation.value_counts())
# df.tail()

# special

In [ ]:
def plot_accident_by_year(df, norm=False, save=False):
    xtab = pd.crosstab(df.event_year, df.Hospitalized)
    xtab.columns = ['Not Hospitalized', 'Hospitalized']
    xtab.columns = ['0','1']
    xtab.columns.name = 'Hospitalized Outcome'
    ylabel = 'Hospitalized Count'
    if norm:
        xtab = xtab.div(xtab.sum(axis=1).astype('float'), axis=0)
        ylabel = 'Hospitalized Percent'
    fig, axes = plt.subplots(1, 1, figsize=(10, 5))
    xtab.plot(kind='bar', ax=axes, stacked=True, alpha=0.5, sort_columns=True)
    axes.set_title('OSHA Accident Investigations')
    axes.set_ylabel(ylabel)
    axes.set_xlabel('Event Year')
#     if save:
#         plt.savefig(structured_dir + 'osha_accident_by_year_plot.png')


In [ ]:

plot_accident_by_year(df, save=True)

In [ ]:
def plot_accident_by_year(df, norm=False, save=False):
    xtab = pd.crosstab(df.event_year, df.Amputation)
    xtab.columns = ['Not Amputation', 'Amputation']
    xtab.columns = ['0','1']
    xtab.columns.name = 'Amputation Outcome'
    ylabel = 'Amputation Count'
    if norm:
        xtab = xtab.div(xtab.sum(axis=1).astype('float'), axis=0)
        ylabel = 'Accident Percent'
    fig, axes = plt.subplots(1, 1, figsize=(10, 5))
    xtab.plot(kind='bar', ax=axes, stacked=True, alpha=0.5, sort_columns=True)
    axes.set_title('OSHA Accident Investigations')
    axes.set_ylabel(ylabel)
    axes.set_xlabel('Event Year')
#     if save:
#         plt.savefig(structured_dir + 'osha_accident_by_year_plot.png')
plot_accident_by_year(df, save=True)

In [ ]:
plot_accident_by_year(df, save=True)

In [ ]:
def plot_accident_by_month(df, norm=False, save=False):
    xtab = pd.crosstab(df.event_month, df.Hospitalized)
#     print(xtab)
    xtab.columns = ['Not Hospitalized', 'Hospitalized']
    xtab.columns = ['0','1']
    xtab.columns.name = 'Hospitalized Outcome'
    ylabel = 'Hospitalized Count'
    if norm:
        xtab = xtab.div(xtab.sum(axis=1).astype('float'), axis=0)
        ylabel = 'Hospitalized Percent'
    fig, axes = plt.subplots(1, 1, figsize=(10, 5))
    xtab.plot(kind='bar', ax=axes, stacked=True, alpha=0.5, sort_columns=True)
    axes.set_title('OSHA Accident Investigations')
    axes.set_ylabel(ylabel)
    axes.set_xlabel('Event Month')
#     if save:
#         plt.savefig(structured_dir + 'osha_accident_by_year_plot.png')
plot_accident_by_month(df, save=True)

In [ ]:
def plot_accident_by_month(df, norm=False, save=False):
    xtab = pd.crosstab(df.event_month, df.Amputation)
    xtab.columns = ['Not Fatal', 'Fatal']
    xtab.columns = ['0','1']
    xtab.columns.name = 'Amputation Outcome'
    ylabel = 'Amputation Count'
    if norm:
        xtab = xtab.div(xtab.sum(axis=1).astype('float'), axis=0)
        ylabel = 'Amputation Percent'
    fig, axes = plt.subplots(1, 1, figsize=(10, 5))
    xtab.plot(kind='bar', ax=axes, stacked=True, alpha=0.5, sort_columns=True)
    axes.set_title('OSHA Accident Investigations')
    axes.set_ylabel(ylabel)
    axes.set_xlabel('Event Month')
#     if save:
#         plt.savefig(structured_dir + 'osha_accident_by_year_plot.png')
plot_accident_by_month(df, save=True)

# DAta scrapping

In [ ]:
df= df[df['event_year'] == 2015]
# df=df.iloc[:100]
# df=df[:9853]
len(df)

In [ ]:
# for i in range(df["Part of Body Title"]):
#     df["Part of Body Title"][i]=df["Part of Body Title"][i].lower()
tit_ind = df[df['Part of Body Title']=='Nonclassifiable' ].index
len(tit_ind)
df.drop(tit_ind , inplace=True)
df["Part of Body Title"]=df["Part of Body Title"].str.replace(r'[^\w\s]+', '')
df["Part of Body Title"]=df["Part of Body Title"].str.replace('unspecified', '')
df["Part of Body Title"] = df["Part of Body Title"].apply(clean_text)

df["EventTitle"]=df["EventTitle"].str.replace(r'[^\w\s]+', '')
df["EventTitle"]=df["Part of Body Title"].str.replace('unspecified', '')
df["EventTitle"] = df["EventTitle"].apply(clean_text)

df["NatureTitle"]=df["NatureTitle"].str.replace(r'[^\w\s]+', '')
df["NatureTitle"]=df["NatureTitle"].str.replace('unspecified', '')
df["NatureTitle"] = df["NatureTitle"].apply(clean_text)

df["SourceTitle"]=df["SourceTitle"].str.replace(r'[^\w\s]+', '')
df["SourceTitle"]=df["Part of Body Title"].str.replace('unspecified', '')
df["SourceTitle"] = df["SourceTitle"].apply(clean_text)



In [ ]:



df

In [ ]:
len(df)
x=set(df['NatureTitle'].values)
len(x)

In [ ]:
import nltk
nltk.download('stopwords') # run this one time

***
Let us pre-process the data
***

In [ ]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df['Part of Body Title']=df['Part of Body Title'].apply(remove_stopwords)
df['EventTitle']=df['EventTitle'].apply(remove_stopwords)
df['NatureTitle']=df['NatureTitle'].apply(remove_stopwords)
df['SourceTitle']=df['SourceTitle'].apply(remove_stopwords)

In [ ]:
df

In [ ]:
# !python3 -m spacy download en_core_web_md 

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])


In [ ]:
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [ ]:

text_list=df['Part of Body Title'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)

# print(tokenized_reviews[1])

In [ ]:
tokenized_text=tokenized_reviews
print(tokenized_text)

***
Create vocabulary dictionary and document term matrix
***

In [ ]:
dictionary = corpora.Dictionary(tokenized_text)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]
print(len(dictionary))


In [ ]:
df['Part of Body Title'].describe()

In [ ]:
df['NatureTitle'].describe()

In [ ]:
df['SourceTitle'].describe()

In [ ]:
df['EventTitle'].describe()

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=3, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [ ]:
lda_model.print_topics(num_words=100)

In [ ]:
x= [lda_model.get_document_topics(item) for item in doc_term_matrix]
x_f=pd.DataFrame(x)
z=[]
for i in x:
    y=[]
    for j in i:
#         print(j[1])
        y.append(j[1])
    z.append(y)
topics=[]
for i in z:
    topics.append(i.index(max(i))+1)
# topics
df['topics_parts']=topics
df

# x.print_topics(num_words=33)

In [ ]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

In [ ]:
doc_term_matrix

In [ ]:
text_list=df['NatureTitle'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
tokenized_text=tokenized_reviews
print(tokenized_text)

dictionary = corpora.Dictionary(tokenized_text)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

LDA = gensim.models.ldamodel.LdaModel



In [ ]:
# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=3, random_state=100,
                chunksize=1000, passes=50,iterations=100)

lda_model.print_topics()



In [ ]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

In [ ]:
x= [lda_model.get_document_topics(item) for item in doc_term_matrix]
x_f=pd.DataFrame(x)
z=[]
for i in x:
    y=[]
    for j in i:
#         print(j[1])
        y.append(j[1])
    z.append(y)
topics=[]
for i in z:
    topics.append(i.index(max(i))+1)
# topics
df['topics_nature']=topics
df

# x.print_topics(num_words=33)

In [ ]:
text_list=df['SourceTitle'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
tokenized_text=tokenized_reviews
print(tokenized_text)

dictionary = corpora.Dictionary(tokenized_text)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

LDA = gensim.models.ldamodel.LdaModel



In [ ]:
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=3, random_state=100,
                chunksize=1000, passes=50,iterations=100)

lda_model.print_topics()
# type(lda_model.print_topics())

In [ ]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

In [ ]:
x= [lda_model.get_document_topics(item) for item in doc_term_matrix]
x_f=pd.DataFrame(x)
z=[]
for i in x:
    y=[]
    for j in i:
#         print(j[1])
        y.append(j[1])
    z.append(y)
topics=[]
for i in z:
    topics.append(i.index(max(i))+1)
# topics
df['topics_source']=topics
df

# x.print_topics(num_words=33)

In [ ]:
text_list=df['EventTitle'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
tokenized_text=tokenized_reviews
print(tokenized_text)

dictionary = corpora.Dictionary(tokenized_text)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

LDA = gensim.models.ldamodel.LdaModel



In [ ]:
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=4, random_state=100,
                chunksize=1000, passes=50,iterations=100)

lda_model.print_topics()


In [ ]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

In [ ]:
x= [lda_model.get_document_topics(item) for item in doc_term_matrix]
x_f=pd.DataFrame(x)
z=[]
for i in x:
    y=[]
    for j in i:
#         print(j[1])
        y.append(j[1])
    z.append(y)
topics=[]
for i in z:
    topics.append(i.index(max(i))+1)
# topics
df['topics_event']=topics
df

# x.print_topics(num_words=33)

# ghxvuqxvuwvyv

In [ ]:
# from sklearn.compose import ColumnTransformer      #it 
# from sklearn.preprocessing import OneHotEncoder   #ifor catogery types , it coverts into binary 
# ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder())],remainder='passthrough')
# #ColTrans(transformers[(kind if trsformer,type of encoding,columns on which we apply)],remainder=).  ->transformers for which we want to apply
# #                                                                                                      remainders.  for which we dont eant to apply
#     #since ct is both type, and next ct.fittrans rdiesnt return as numpy

 


In [ ]:
df.head(10)

In [ ]:

y=pd.DataFrame()
y['Amputation']=df['Amputation']
y['Hospitalized']=df['Hospitalized']
y

# z=df['Amputation'].values
# z=z.reshape(len(z),1)
# y=df['Hospitalized'].values
# y=y.reshape(len(y),1)

# y=np.append(y,z,axis=1)
# # y=y[:1000]
# y.shape
# y=pd.DataFrame(y,columns=['Hospitalized','Amputation'])



In [ ]:
# x=df[:1000]
x=pd.DataFrame()
x['Amputation']=df['Amputation']
x['Hospitalized']=df['Hospitalized']
x['topics_parts']=df['topics_parts']
x['topics_event']=df['topics_event']
x['topics_source']=df['topics_source']
x['topics_nature']=df['topics_nature']

# x=x.drop(['IDUPA', 'EventDate','Nature','Part of Body','Event','Source','Hospitalized','Amputation','time_stamp','event_month','event_year','event_weekday'], axis=1)

x

In [ ]:
y=x[x['topics_source']==1]
len(y)

In [ ]:
y=x[x['topics_source']==2]
len((y))

In [ ]:
y=x[x['topics_source']==3]
len((y))

In [ ]:
y=x[x['topics_event']==4]
len((y))

In [ ]:

ohe = pd.get_dummies(data=x, columns=['topics_parts','topics_event','topics_source','topics_nature'])
z=pd.DataFrame(ohe)
# z=z.iloc[:,:100]
z

In [ ]:
z

In [ ]:
z.dtypes

In [ ]:
y_=y.iloc[:,:1]

# z["Hospitalized"] = z["Hospitalized"].astype(int)
# z["Amputation"] = z["Amputation"].astype(int)


z["Hospitalized"] = pd.to_numeric(z["Hospitalized"])
z["Amputation"] = pd.to_numeric(z["Amputation"])


# qf=pd.concat([z,y], axis=1, join='inner')
# qf.dtypes
# qf['NatureTitle_abdominal pain'].value_counts()

In [ ]:
z.info()

In [ ]:
z['Amputation'].value_counts()

In [ ]:
qf=z

In [ ]:
from sklearn.model_selection import train_test_split
train , test=train_test_split(qf,test_size=0.3,random_state=42)
# # q=q.transpose()

# train_X, test_X, train_y, test_y = train_test_split(q, y_, train_size = 0.3, random_state = 0)
train

# Amputation

In [ ]:
print(len(train))
print(len(train.iloc[0]))
train_X=train.iloc[:,2:]
train_y=train.iloc[:,0]
test_X=test.iloc[:,2:]
test_y=test.iloc[:,0]

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()


# scaler = MinMaxScaler()
# model=scaler.fit(data)
# scaled_data=model.transform(data)

train_X = pd.DataFrame(sc.fit_transform(train_X))
test_X = pd.DataFrame(sc.transform(test_X))

# train_X = scaler.fit_transform(columns=train_X.columns)
# test_X = scaler.fit_transform(columns=test_X.columns)

In [ ]:
np.isnan(train_y.any())

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
train_X,train_y  = sm.fit_resample(train_X, train_y)
test_X,test_y  = sm.fit_resample(test_X, test_y)

In [ ]:
train_y.value_counts()

In [ ]:


from sklearn.preprocessing import StandardScaler
# # import seaborn as sns
 
# std_scaler = StandardScaler()
 
# train_X = std_scaler.fit_transform(train_X)
# df_scaled = pd.DataFrame(df_scaled, columns=train_X.columns)
 
# print("Scaled Dataset Using StandardScaler")
# X_train=df_scaled


# 
# train_X=(train_X-train_X.mean())/train_X.std()

In [ ]:

test_y.columns=['Hospitalized']
test_y
# print(test_y.Hospitalized.value_counts())
# print()
print(type(test_y))
# print(test_y.Amputation.value_counts())
# y.info

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [ ]:
train_X

In [ ]:
test_y

In [ ]:
model=LogisticRegression()
model.fit(train_X,train_y)
model_pred=model.predict(test_X)
print("Accuarcy=",accuracy_score(model_pred,test_y))
print('class report is')
print(classification_report(model_pred,test_y))
print(roc_auc_score(test_y, model_pred))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(test_y, model_pred)


In [ ]:
model=SVC()
model.fit(train_X,train_y)
model_pred=model.predict(test_X)
print("Accuracy=",accuracy_score(model_pred,test_y))
print('classf report is')
print(classification_report(model_pred,test_y))
print(roc_auc_score(test_y, model_pred))

In [ ]:

model=KNeighborsClassifier()
model.fit(train_X,train_y)
model_pred=model.predict(test_X)
print("Accuracy",accuracy_score(model_pred,test_y))
print('classf report is')
print(classification_report(model_pred,test_y))
print(roc_auc_score(test_y, model_pred))

In [ ]:
# a_index=list(range(1,11))
# a=pd.Series()
# x=[1,2,3,4,5,6,7,8,9,10]
# for i in list(range(1,11)):
#     model=KNeighborsClassifier(n_neighbors=i) 
#     model.fit(train_X,train_y)
#     prediction=model.predict(test_X)
#     a=a.append(pd.Series(accuracy_score(prediction,test_y)))
# plt.plot(a_index, a)
# plt.xticks(x)



In [ ]:
mod=MultinomialNB()
mod.fit(train_X,train_y) 
model_pred=mod.predict(test_X) 
print('The accuracy is:',accuracy_score(model_pred,test_y))
print("The classification report is")
print(classification_report(model_pred,test_y))
print(roc_auc_score(test_y, model_pred))

In [ ]:
# model = RandomForestClassifier(class_weight='balanced')
model = RandomForestClassifier()
model.fit(train_X,train_y) 
prediction=model.predict(test_X) 
print('The accuracy =',accuracy_score(prediction,test_y))
print("The classification report is")
print(classification_report(prediction,test_y))
print(roc_auc_score(test_y, prediction))

In [ ]:
# from matplotlib import pyplot as plt
# from sklearn.metrics import confusion_matrix , classification_report
# def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
#     df_cm = pd.DataFrame(
#         confusion_matrix, index=class_names, columns=class_names, 
#     )
#     fig = plt.figure(figsize=figsize)
#     try:
#         heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
#     except ValueError:
#         raise ValueError("Confusion matrix values must be integers.")
#     heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
#     heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
#     plt.ylabel('Truth')
#     plt.xlabel('Prediction')
# cm = confusion_matrix(prediction, test_y)
# print_confusion_matrix(cm,['is','not'])
# # accuracy_score(y_test, y_pred)

In [ ]:
model = DecisionTreeClassifier()
model.fit(train_X,train_y) 
prediction=model.predict(test_X) 
print('The accuracy is:',accuracy_score(prediction,test_y))
print("The classification report is")
print(classification_report(prediction,test_y))
print(roc_auc_score(test_y, prediction))

In [ ]:
model = XGBClassifier()
model.fit(train_X,train_y) 
prediction=model.predict(test_X) 
print('The accuracy is:',accuracy_score(prediction,test_y))
print("The classification report is")
print(classification_report(prediction,test_y))
print(roc_auc_score(test_y, prediction))

In [ ]:
train_X=train.iloc[:,2:]
train_y=train.iloc[:,1]
test_X=test.iloc[:,2:]
test_y=test.iloc[:,1]
train_X.shape

In [ ]:
test_y.value_counts()

# Hospitalization

In [ ]:
print(len(train))
print(len(train.iloc[0]))
train_X=train.iloc[:,2:]
train_y=train.iloc[:,1]
test_X=test.iloc[:,2:]
test_y=test.iloc[:,1]

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()

# train_X = pd.DataFrame(sc.fit_transform(X_train))
# test_X = pd.DataFrame(sc.transform(X_test))

print(train_X)


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
train_X,train_y  = sm.fit_resample(train_X, train_y)
test_X,test_y  = sm.fit_resample(test_X, test_y)

In [ ]:

test_y.columns=['Hospitalized']
test_y
# print(test_y.Hospitalized.value_counts())
# print()
print(type(test_y))
# print(test_y.Amputation.value_counts())
# y.info

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score,roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
test_y

In [ ]:
model=LogisticRegression()
model.fit(train_X,train_y)
model_pred=model.predict(test_X)
print("Accuarcy=",accuracy_score(model_pred,test_y))
print('class report is')
print(classification_report(model_pred,test_y))
print(precision_score(model_pred,test_y))
print(recall_score(test_y, model_pred))
print(roc_auc_score(test_y, model_pred))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(test_y, model_pred)

In [ ]:
model=SVC()
model.fit(train_X,train_y)
model_pred=model.predict(test_X)
print("Accuracy=",accuracy_score(model_pred,test_y))
print('classf report is')
print(classification_report(model_pred,test_y))
print(precision_score(model_pred,test_y))
print(recall_score(test_y, model_pred))
print(roc_auc_score(test_y, model_pred))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(test_y, model_pred)

In [ ]:
model=KNeighborsClassifier()
model.fit(train_X,train_y)
model_pred=model.predict(test_X)
print("Accuracy",accuracy_score(model_pred,test_y))
print('classf report is')
print(classification_report(model_pred,test_y))
print(roc_auc_score(model_pred,test_y))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(test_y, model_pred)

In [ ]:
# a_index=list(range(1,11))
# a=pd.Series()
# x=[1,2,3,4,5,6,7,8,9,10]
# for i in list(range(1,11)):
#     model=KNeighborsClassifier(n_neighbors=i) 
#     model.fit(train_X,train_y)
#     prediction=model.predict(test_X)
#     a=a.append(pd.Series(accuracy_score(prediction,test_y)))
# plt.plot(a_index, a)
# plt.xticks(x)



In [ ]:
mod=MultinomialNB()
mod.fit(train_X,train_y) 
model_pred=mod.predict(test_X) 
print('The accuracy is:',accuracy_score(model_pred,test_y))
print("The classification report is")
print(classification_report(model_pred,test_y))
print(roc_auc_score(model_pred,test_y))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(test_y, model_pred)

In [ ]:
# model = RandomForestClassifier(class_weight='balanced')
model = RandomForestClassifier()
model.fit(train_X,train_y) 
prediction=model.predict(test_X) 
print('The accuracy =',accuracy_score(prediction,test_y))
print("The classification report is")
print(classification_report(prediction,test_y))
print(roc_auc_score(prediction,test_y))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(prediction, test_y)

In [ ]:
model = DecisionTreeClassifier()
model.fit(train_X,train_y) 
prediction=model.predict(test_X) 
print('The accuracy is:',accuracy_score(prediction,test_y))
print("The classification report is")
print(classification_report(prediction,test_y))
print(roc_auc_score(prediction,test_y))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix , classification_report
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
cm = confusion_matrix(model_pred, test_y)
print_confusion_matrix(cm,['is','not'])
accuracy_score(prediction, test_y)

In [ ]:
model = XGBClassifier()
model.fit(train_X,train_y) 
prediction=model.predict(test_X) 
print('The accuracy is:',accuracy_score(prediction,test_y))
print("The classification report is")
print(classification_report(prediction,test_y))
print(roc_auc_score(prediction,test_y))

In [ ]:

print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


***
Method to find optimal number of topics
Code from:https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#14computemodelperplexityandcoherencescore
***

In [ ]:


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [ ]:
# Show graph
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores


In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[7]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, doc_term_matrix, dictionary)
vis

In [ ]:
x='2'
type(x)